In [12]:
import cv2
import os
import numpy as np
import tensorflow as tf
import keras
from sklearn.mixture import GaussianMixture
from sklearn import mixture
from scipy.spatial import distance
from tqdm.notebook import tqdm
import seaborn
import matplotlib.pyplot as plt
import time


def load_images_from_folder(folder):
    images = []
    for filename in tqdm(sorted(os.listdir(folder))):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        if img is not None:
            images.append(img)
    return np.array(images)

path = load_images_from_folder("/home/ws575033/thesis_nb/Train_fin")
path.shape

start = time.time()

Xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling=None,
    #classes=1000,
    #classifier_activation="softmax",
)


features = Xception.predict(
    path,
    batch_size=None,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
features.shape
features_linear = features.reshape(-1, features.shape[-1])
features_linear.shape

from sklearn.svm import OneClassSVM
svm = OneClassSVM(gamma='scale').fit(features_linear)

test_image = load_images_from_folder("/home/ws575033/thesis_nb/Data_set/grid_values/trolley")
dist_list=[]
for i in range(len(test_image)):
    tes = test_image[i]
    tes = np.expand_dims(tes, axis=0)
    features2 = Xception.predict(
    tes,
    batch_size=None,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    )

    
    features2_linear = features2.reshape(-1, features2.shape[-1])
 
    maha = svm.decision_function(features2_linear) 
    print(i, "mahalonobis distance ------------------------------>>>>>>>", max(abs(maha)))
    
    end = time.time()
    print(f"Runtime of the program is {end - start}")
    
    dist_list.append(maha.max())
dist_list = np.array(dist_list)
dist_list.shape

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 mahalonobis distance ------------------------------>>>>>>> 325.85833214767206
Runtime of the program is 28.65111780166626


(1,)